# Source detction

In [ ]:
import numpy as np
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from astroquery.sdss import SDSS

In [ ]:
from astropy.io import fits
from astropy import wcs
fits_image_filename = '../Data/newfits/calexp-1-108438.fits'

hdu_list = fits.open(fits_image_filename)
hdu = fits.open(fits_image_filename)[1]
w = wcs.WCS(hdu.header)

In [ ]:
image_data = fits.getdata(fits_image_filename,0)
print(type(image_data))
print(image_data.shape)
# Parse the WCS keywords in the primary HDU
print(w)

In [ ]:
from astropy.convolution import Gaussian2DKernel
from photutils import Background2D, MedianBackground
from photutils import detect_threshold, detect_sources
data = image_data
bkg_estimator = MedianBackground()
bkg = Background2D(data, (50, 50), filter_size=(3, 3), bkg_estimator=bkg_estimator)
threshold = bkg.background + (10. * bkg.background_rms)

In [ ]:
from astropy.stats import gaussian_fwhm_to_sigma
sigma = 3.0 * gaussian_fwhm_to_sigma  # FWHM = 3.
kernel = Gaussian2DKernel(sigma, x_size=3, y_size=3)
kernel.normalize()
npixels = 5
segm = detect_sources(data, threshold, npixels=npixels,filter_kernel=kernel)
#segm_deblend = deblend_sources(data, segm, npixels=npixels,filter_kernel=kernel, nlevels=32,contrast=0.001)

In [ ]:
from photutils import source_properties
cat = source_properties(data, segm, wcs=w)
df = tbl.to_pandas()

In [ ]:
indexNames = df[df['xcentroid'] < 400].index
dfsel = df.drop(indexNames)
indexNames = dfsel[dfsel['xcentroid'] > 7776].index
dfsel = dfsel.drop(indexNames)
indexNames = dfsel[dfsel['ycentroid'] < 300].index
dfsel = dfsel.drop(indexNames)
indexNames = dfsel[dfsel['ycentroid'] > 5832].index
dfsel = dfsel.drop(indexNames)
indexNames = dfsel[dfsel['elongation'] > 10].index
dfsel = dfsel.drop(indexNames)
print(dfsel)
dfsel = dfsel[['xcentroid','ycentroid','sky_centroid.ra','sky_centroid.dec','elongation','equivalent_radius','area']]

In [ ]:
coords = list(zip(dfsel.xcentroid, dfsel.ycentroid))
print(coords)
dfsel['coords'] = coords

# Cross Referencing

In [ ]:
lon1, lat1 = w.all_pix2world(0, 0, 0)
lon2, lat2 = w.all_pix2world(6132, 8176, 0)
maxlon = max(lon1,lon2)
minlon = min(lon1,lon2)
maxlat = max(lat1,lat2)
minlat = min(lat1,lat2)

In [ ]:
query = f"""
SELECT ra,dec
FROM Galaxy
WHERE ra between {minlon} and {maxlon}
AND dec between {minlat} and {maxlat}
AND g < 20
"""
res = SDSS.query_sql(query)
galaxies = res.to_pandas()

In [ ]:
query = f"""
SELECT ra,dec
FROM Star
WHERE ra between {minlon} and {maxlon}
AND dec between {minlat} and {maxlat}
AND g < 20
"""
res = SDSS.query_sql(query)
stars = res.to_pandas()

In [12]:
def threesecapprox(x,y):
    """
    compares if x and y are within 3 arcseconds of each other
    
    Arguments:
    x (float): one value in degrees
    
    y (float): another value in degrees
    """
    truth = x*206265 > y*206265-3 and x*206265 < y*206265+3
    
    return truth

# Cutting out

In [ ]:
# Make the cutout, including the WCS
cutout = Cutout2D(image_data, position=coords[1000], size=size, wcs=w)

# Put the cutout image in the FITS HDU
hdu = fits.PrimaryHDU(cutout.data)

# Update the FITS header with the cutout WCS
hdu.header.update(cutout.wcs.to_header(relax=False))


# Write the cutout to a new FITS file
cutout_filename = '004.fits'
hdu.writeto(cutout_filename, overwrite=True)